In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from utils import create_sequences

2024-01-23 06:22:08.203736: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 06:22:08.203792: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 06:22:08.203814: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 06:22:08.208859: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('./dataset/anomaly_dataset.csv').set_index('timestamp')
max_val = np.max(df.values)

seq_data = create_sequences(df, seq_len=24)
seq_data /= max_val

In [3]:
train = seq_data[:(365 * 24 * 3)]
val = seq_data[(365 * 24 * 3):(365 * 24 * 4)]
test = seq_data[(365 * 24 * 4):]

print(f'train data: {len(train)}')
print(f'validation data: {len(val)}')
print(f'test data: {len(test)}')

train data: 26280
validation data: 8760
test data: 3601


In [11]:
# 1(잔차 연결 x)
class TCN(layers.Layer):
    def __init__(self, output_dim, kernel_size, dilations:list):
        super(TCN, self).__init__()
        self.tcns = [layers.Conv1D(output_dim, kernel_size, padding='causal', dilation_rate=dilation, activation='relu') for dilation in dilations]

    def call(self, x):
        for tcn in self.tcns:
            x = tcn(x)
        return x

In [12]:
class Encoder(models.Model):

    def __init__(self, seq_length, latent_dim):
        super(Encoder, self).__init__()

        self.h1 = TCN(128, kernel_size=3, dilations=[1, 2, 4, 8])  # (seq_len, input_dim) -> (seq_len, 128))
        self.h2 = layers.Dense(latent_dim) # (seq_len , 128) -> (latent_dim)
        self.h3 = layers.RepeatVector(seq_length) # (latent_dim) -> (seq_length, latent_dim)

    def call(self, x):
        x = self.h1(x)
        x = layers.Flatten()(x)
        z = self.h2(x)
        z_rep = self.h3(z)

        return z, z_rep


class Decoder(models.Model):

    def __init__(self, input_dim, latent_dim):
        super(Decoder, self).__init__()

        # self.h1 = layers.Conv(latent_dim, return_sequences=True) # (seq_length, latent_dim) -> (seq_len, input_dim)
        self.h1 = TCN(128, kernel_size=3, dilations=[1, 2, 4, 8]) # (seq_len, input_dim) -> (seq_length, 128)
        self.h2 = layers.TimeDistributed(layers.Dense(input_dim)) # (seq_length, 128) -> (seq_length, input_dim)

    def call(self, x):
        x = self.h1(x)
        x = self.h2(x)
        # x = self.h3(x)

        return x



class LstmAE(models.Model):

    def __init__(self, seq_length, input_dim, latent_dim):
        super(LstmAE, self).__init__()

        self.encoder = Encoder(seq_length, latent_dim)
        self.decoder = Decoder(input_dim, latent_dim)

    def call(self, x):
        z, z_rep = self.encoder(x)
        decoded = self.decoder(z_rep)

        return decoded

In [13]:
model = LstmAE(seq_length=24, input_dim=1, latent_dim=64)
model.build(input_shape=(None, train.shape[1], train.shape[2]))
model.summary()

Model: "lstm_ae_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_2 (Encoder)         multiple                  345024    
                                                                 
 decoder (Decoder)           multiple                  172673    
                                                                 
Total params: 517697 (1.97 MB)
Trainable params: 517697 (1.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
optim = optimizers.Adam(learning_rate=.001)
model.compile(optimizer=optim, loss='mse')

In [15]:
history = model.fit(train, train, epochs=50, batch_size=64, validation_data=(val, val))

Epoch 1/50


2024-01-23 06:31:06.884612: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-01-23 06:31:07.728329: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7faa34005bb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-23 06:31:07.728368: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-01-23 06:31:07.732382: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-23 06:31:07.807282: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


411/411 [==============================] - 9s 14ms/step - loss: 0.0055 - val_loss: 9.6644e-04
Epoch 2/50
411/411 [==============================] - 4s 10ms/step - loss: 6.4589e-04 - val_loss: 4.2490e-04
Epoch 3/50
411/411 [==============================] - 5s 11ms/step - loss: 3.6521e-04 - val_loss: 3.4356e-04
Epoch 4/50
411/411 [==============================] - 5s 12ms/step - loss: 3.0872e-04 - val_loss: 2.3587e-04
Epoch 5/50
411/411 [==============================] - 5s 12ms/step - loss: 1.8554e-04 - val_loss: 2.7871e-04
Epoch 6/50
411/411 [==============================] - 5s 12ms/step - loss: 1.8838e-04 - val_loss: 1.4680e-04
Epoch 7/50
411/411 [==============================] - 5s 12ms/step - loss: 1.8909e-04 - val_loss: 1.4672e-04
Epoch 8/50
411/411 [==============================] - 5s 12ms/step - loss: 1.4736e-04 - val_loss: 2.0228e-04
Epoch 9/50
411/411 [==============================] - 5s 12ms/step - loss: 1.3296e-04 - val_loss: 7.5305e-05
Epoch 10/50
411/411 [=============

In [16]:
model.save('./models/new_TCNAE/new_TCNAE')

with open('./models/new_TCNAE/new_TCNAE_hist.pkl', 'wb') as f:
    pickle.dump(history.history, f)

INFO:tensorflow:Assets written to: ./models/new_TCNAE/new_TCNAE/assets


INFO:tensorflow:Assets written to: ./models/new_TCNAE/new_TCNAE/assets
